In [35]:
# pip install graphene==3.0.0
# pip install mysql-connector-python

# IMPORT LIBRARIES

In [36]:
import graphene
from pprint import pprint
import mysql.connector
from mysql.connector import errorcode


# AIVEN DB CONNECTION 

## Connect databas

In [37]:
config = {
    'user': 'avnadmin',
    'password': 'AVNS_8pk8_VpEMvETZuR__I4',
    'host': 'libratur-database-xylamaharanii-9ca8.b.aivencloud.com',
    'port': '15532',
    'database': 'graphql_eai',
}

conn = mysql.connector.connect(**config)

### Checking connection

In [38]:
try:
    conn = mysql.connector.connect(**config)
    print("Connection successful")
    cursor = conn.cursor()
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

Connection successful


## Create Table

In [39]:
# create_table_query = """
#     CREATE TABLE IF NOT EXISTS tiket_penerbangan (
#         kodePenerbangan INT PRIMARY KEY,
#         namaMaskapai VARCHAR(255) NOT NULL,
#         harga INT NOT NULL
#     )
#     """
# cursor.execute(create_table_query)
# conn.commit()

In [40]:
# import mysql.connector
# from mysql.connector import errorcode

# # Aiven DB Connection
# config = {
#     'user': 'avnadmin',
#     'password': 'AVNS_8pk8_VpEMvETZuR__I4',
#     'host': 'libratur-database-xylamaharanii-9ca8.b.aivencloud.com',
#     'port': '15532',
#     'database': 'graphql_eai',
# }

# # Establish connection
# try:
#     conn = mysql.connector.connect(**config)
#     print("Connection successful")
#     cursor = conn.cursor()

#     # Create keranjang table
#     create_keranjang_table_query = """
#     CREATE TABLE IF NOT EXISTS keranjang (
#         id INT AUTO_INCREMENT PRIMARY KEY,
#         kodePenerbangan INT,
#         namaPemesan VARCHAR(255),
#         jumlahTiket INT,
#         FOREIGN KEY (kodePenerbangan) REFERENCES tiket_penerbangan(kodePenerbangan)
#     )
#     """
#     cursor.execute(create_keranjang_table_query)
#     conn.commit()
#     print("keranjang table created successfully")



### create view for keranjang

In [41]:
# # Establish connection
# try:
#     conn = mysql.connector.connect(**config)
#     print("Connection successful")
#     cursor = conn.cursor()

#     # Create view to calculate total price
#     create_view_query = """
#     CREATE OR REPLACE VIEW keranjang_view AS
#     SELECT 
#         k.id,
#         k.kodePenerbangan,
#         k.namaPemesan,
#         k.jumlahTiket,
#         tp.harga,
#         (k.jumlahTiket * tp.harga) AS totalPrice
#     FROM 
#         keranjang k
#     JOIN 
#         tiket_penerbangan tp ON k.kodePenerbangan = tp.kodePenerbangan
#     """
#     cursor.execute(create_view_query)
#     conn.commit()
#     print("View keranjang_view created successfully")

#     cursor.close()
# except mysql.connector.Error as err:
#     if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
#         print("Something is wrong with your username or password")
#     elif err.errno == errorcode.ER_BAD_DB_ERROR:
#         print("Database does not exist")
#     else:
#         print(err)



## Add Data

In [42]:
# ## adding data
# data = [
#         {'kodePenerbangan': 1, 'namaMaskapai': 'Indonesia Air', 'harga': 2500000},
#         {'kodePenerbangan': 2, 'namaMaskapai': 'Australia Air', 'harga': 2600000},
#         {'kodePenerbangan': 3, 'namaMaskapai': 'Netherland Air', 'harga': 2700000},
#         {'kodePenerbangan': 4, 'namaMaskapai': 'America Air', 'harga': 2800000},
#         {'kodePenerbangan': 5, 'namaMaskapai': 'Missisipi Air', 'harga': 2900000},
#         {'kodePenerbangan': 6, 'namaMaskapai': 'Japan Air', 'harga': 3000000}
#     ]
    
# insert_query = """
#     INSERT INTO tiket_penerbangan (kodePenerbangan, namaMaskapai, harga)
#     VALUES (%s, %s, %s)
#     """
    
# for item in data:
#         cursor.execute(insert_query, (item['kodePenerbangan'], item['namaMaskapai'], item['harga']))
    
# conn.commit()
# cursor.close()
# conn.close()
# print("Table created and data inserted successfully")

# OBJECT, QUERY, and MUTATION

In [43]:
class TiketPenerbangan(graphene.ObjectType):
    kodePenerbangan = graphene.ID()
    namaMaskapai = graphene.String()
    harga = graphene.Int()

class Keranjang(graphene.ObjectType):
    id = graphene.ID()
    kodePenerbangan = graphene.Int()
    namaPemesan = graphene.String()
    jumlahTiket = graphene.Int()
    harga = graphene.Int()
    totalPrice = graphene.Int()




# Input Types for Mutations
class AddTiketInput(graphene.InputObjectType):
    kodePenerbangan = graphene.Int(required=True)
    namaMaskapai = graphene.String(required=True)
    harga = graphene.Int(required=True)

class AddToKeranjangInput(graphene.InputObjectType):
    kodePenerbangan = graphene.Int(required=True)
    namaPemesan = graphene.String(required=True)
    jumlahTiket = graphene.Int(required=True)

class UpdateKeranjangInput(graphene.InputObjectType):
    id = graphene.ID(required=True)
    namaPemesan = graphene.String(required=True)
    kodePenerbangan = graphene.Int(required=True)
    jumlahTiket = graphene.Int(required=True)





## MUTATION TO INSERT NEW FLIGHT
class AddTiket(graphene.Mutation):
    class Arguments:
        input = AddTiketInput(required=True)

    tiket = graphene.Field(TiketPenerbangan)

    def mutate(self, info, input):
        try:
            conn = mysql.connector.connect(**config)
            cursor = conn.cursor()

            # Validate input
            if input.kodePenerbangan < 0:
                raise ValueError("Kode Penerbangan must be a positive integer.")
            if input.harga <= 0:
                raise ValueError("Harga must be a positive integer.")

            add_tiket_query = """
            INSERT INTO tiket_penerbangan (kodePenerbangan, namaMaskapai, harga)
            VALUES (%s, %s, %s)
            """
            cursor.execute(add_tiket_query, (input.kodePenerbangan, input.namaMaskapai, input.harga))
            conn.commit()
            cursor.close()
            conn.close()

            return AddTiket(tiket=TiketPenerbangan(kodePenerbangan=input.kodePenerbangan, namaMaskapai=input.namaMaskapai, harga=input.harga))
        except mysql.connector.Error as err:
            print("Error:", err)
            return None
        except ValueError as ve:
            print("Validation Error:", ve)
            return None




# MUTATION TO INSERT TO KERANJANG
class AddToKeranjang(graphene.Mutation):
    class Arguments:
        input = AddToKeranjangInput(required=True)

    keranjang = graphene.Field(Keranjang)

    def mutate(self, info, input):
        try:
            conn = mysql.connector.connect(**config)
            cursor = conn.cursor(dictionary=True)

            # Validate input
            if input.kodePenerbangan < 0:
                raise ValueError("Kode Penerbangan must be a positive integer.")
            if input.jumlahTiket <= 0:
                raise ValueError("Jumlah Tiket must be a positive integer.")

            # Get the price of the flight from tiket_penerbangan
            cursor.execute("SELECT harga FROM tiket_penerbangan WHERE kodePenerbangan = %s", (input.kodePenerbangan,))
            flight = cursor.fetchone()

            if not flight:
                raise ValueError("Flight not found")

            totalPrice = flight['harga'] * input.jumlahTiket

            # Insert into keranjang
            add_keranjang_query = """
            INSERT INTO keranjang (namaPemesan, kodePenerbangan, jumlahTiket, totalPrice)
            VALUES (%s, %s, %s, %s)
            """
            cursor.execute(add_keranjang_query, (input.namaPemesan, input.kodePenerbangan, input.jumlahTiket, totalPrice))
            conn.commit()

            keranjang_id = cursor.lastrowid

            # Query the view to get the total price
            cursor.execute("SELECT * FROM keranjang_view WHERE id = %s", (keranjang_id,))
            keranjang_item = cursor.fetchone()

            cursor.close()
            conn.close()

            if keranjang_item:
                return AddToKeranjang(keranjang=Keranjang(
                    id=keranjang_item['id'],
                    namaPemesan=keranjang_item['namaPemesan'],
                    kodePenerbangan=keranjang_item['kodePenerbangan'],
                    jumlahTiket=keranjang_item['jumlahTiket'],
                    totalPrice=keranjang_item['totalPrice']
                ))
            else:
                raise ValueError("Keranjang item not found")
        except mysql.connector.Error as err:
            print("Database Error:", err)
            return None
        except ValueError as ve:
            print("Validation Error:", ve)
            return None






## MUTATION TO UPDATE KERANJANG
class UpdateKeranjang(graphene.Mutation):
    class Arguments:
        input = UpdateKeranjangInput(required=True)

    keranjang = graphene.Field(Keranjang)

    def mutate(self, info, input):
        try:
            cursor.execute("SELECT * FROM keranjang WHERE id = %s", (input.id,))
            keranjang_item = cursor.fetchone()
            if not keranjang_item:
                raise ValueError("Keranjang item not found")

            # Update the keranjang item
            update_keranjang_query = """
            UPDATE keranjang
            SET namaPemesan = %s, kodePenerbangan = %s, jumlahTiket = %s
            WHERE id = %s
            """
            cursor.execute(update_keranjang_query, (input.namaPemesan, input.kodePenerbangan, input.jumlahTiket, input.id))
            conn.commit()

            # Get the updated item
            cursor.execute("SELECT * FROM keranjang WHERE id = %s", (input.id,))
            updated_item = cursor.fetchone()

            cursor.close()
            conn.close()

            return UpdateKeranjang(keranjang=Keranjang(**updated_item))

        except mysql.connector.Error as err:
            print("Error:", err)
            return None



class Query(graphene.ObjectType):
    tiket = graphene.Field(TiketPenerbangan, kodePenerbangan=graphene.ID(required=True))
    tikets = graphene.List(TiketPenerbangan)
    keranjang_items = graphene.List(Keranjang)

    def resolve_keranjang_items(self, info):
        try:
            conn = mysql.connector.connect(**config)
            cursor = conn.cursor(dictionary=True)

            # Query the view
            query_view = "SELECT * FROM keranjang_view"
            cursor.execute(query_view)
            results = cursor.fetchall()

            cursor.close()
            conn.close()

            return [Keranjang(**result) for result in results]
        except mysql.connector.Error as err:
            print("Error:", err)
            return []

    def resolve_tiket(self, info, kodePenerbangan):
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM tiket_penerbangan WHERE kodePenerbangan = %s", (kodePenerbangan,))
        result = cursor.fetchone()
        cursor.close()
        if result:
            return TiketPenerbangan(**result)
        return None

    def resolve_tikets(self, info):
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM tiket_penerbangan")
        results = cursor.fetchall()
        cursor.close()
        return [TiketPenerbangan(**result) for result in results]

    def resolve_add_to_keranjang(self, info, input):
        try:
            conn = mysql.connector.connect(**config)
            cursor = conn.cursor(dictionary=True)

            # Validate input
            if input.kodePenerbangan < 0:
                raise ValueError("Kode Penerbangan must be a positive integer.")
            if input.jumlahTiket <= 0:
                raise ValueError("Jumlah Tiket must be a positive integer.")

            # Get the price of the flight from tiket_penerbangan
            cursor.execute("SELECT harga FROM tiket_penerbangan WHERE kodePenerbangan = %s", (input.kodePenerbangan,))
            flight = cursor.fetchone()

            if not flight:
                raise ValueError("Flight not found")

            totalPrice = flight['harga'] * input.jumlahTiket

            # Insert into keranjang
            add_keranjang_query = """
            INSERT INTO keranjang (namaPemesan, kodePenerbangan, jumlahTiket, totalPrice)
            VALUES (%s, %s, %s, %s)
            """
            cursor.execute(add_keranjang_query, (input.namaPemesan, input.kodePenerbangan, input.jumlahTiket, totalPrice))
            conn.commit()

            keranjang_id = cursor.lastrowid

            cursor.close()
            conn.close()

            return AddToKeranjang(keranjang=Keranjang(
                id=keranjang_id,
                namaPemesan=input.namaPemesan,
                kodePenerbangan=input.kodePenerbangan,
                jumlahTiket=input.jumlahTiket,
                totalPrice=totalPrice
            ))
        except mysql.connector.Error as err:
            print("Error:", err)
            return None


# Define Mutations
class Mutation(graphene.ObjectType):
    add_tiket = AddTiket.Field()
    add_to_keranjang = AddToKeranjang.Field()
    update_keranjang = UpdateKeranjang.Field()

# Create Schema
schema = graphene.Schema(query=Query, mutation=Mutation)

# SHOW INFORMATION

## Get Info by ID

In [44]:
q = """
{
    tiket(kodePenerbangan: "8") {
        kodePenerbangan
        namaMaskapai        
        harga
    }
}
"""

result = schema.execute(q)
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Result data:")
    pprint(result.data)

Result data:
{'tiket': {'harga': 9999999,
           'kodePenerbangan': '8',
           'namaMaskapai': 'Testing Air'}}


## Get all Info

In [45]:
q = """
{
    tikets {
        kodePenerbangan  
        harga
        namaMaskapai
    }
}
"""

result = schema.execute(q)
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Result data:")
    pprint(result.data)

Result data:
{'tikets': [{'harga': 2500000,
             'kodePenerbangan': '1',
             'namaMaskapai': 'Indonesia Air'},
            {'harga': 2600000,
             'kodePenerbangan': '2',
             'namaMaskapai': 'Australia Air'},
            {'harga': 2700000,
             'kodePenerbangan': '3',
             'namaMaskapai': 'Netherland Air'},
            {'harga': 2800000,
             'kodePenerbangan': '4',
             'namaMaskapai': 'America Air'},
            {'harga': 2900000,
             'kodePenerbangan': '5',
             'namaMaskapai': 'Missisipi Air'},
            {'harga': 3000000,
             'kodePenerbangan': '6',
             'namaMaskapai': 'Japan Air'},
            {'harga': 3100000,
             'kodePenerbangan': '7',
             'namaMaskapai': 'Korean Air'},
            {'harga': 9999999,
             'kodePenerbangan': '8',
             'namaMaskapai': 'Testing Air'},
            {'harga': 888888888,
             'kodePenerbangan': '9',
       

# INSERT FLIGHTS

## Add Flights

In [46]:
mutation_add_tiket = """
mutation {
    addTiket(input: {kodePenerbangan: -1, namaMaskapai: "Lion Air", harga: 1222222}) {
        tiket {
            kodePenerbangan
            namaMaskapai
            harga
        }
    }
}
"""

result = schema.execute(mutation_add_tiket)
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Mutation result data:")
    pprint(result.data)


Validation Error: Kode Penerbangan must be a positive integer.
Mutation result data:
{'addTiket': None}


## Add to Keranjang

In [47]:
mutation_add_to_keranjang = """
mutation {
    addToKeranjang(input: {namaPemesan: "Regina", kodePenerbangan: 30, jumlahTiket: 3}) {
        keranjang {
            id
            namaPemesan
            kodePenerbangan
            jumlahTiket
            totalPrice
        }
    }
}
"""

result = schema.execute(mutation_add_to_keranjang)
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Mutation result data:")
    pprint(result.data)

Validation Error: Flight not found
Mutation result data:
{'addToKeranjang': None}


### Show Keranjang

In [48]:
query_keranjang_items = """
{
    keranjangItems {
        id
        kodePenerbangan
        namaPemesan
        jumlahTiket
        harga
        totalPrice
    }
}
"""

result = schema.execute(query_keranjang_items)
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Query result data:")
    pprint(result.data)

Query result data:
{'keranjangItems': [{'harga': 2500000,
                     'id': '1',
                     'jumlahTiket': 2,
                     'kodePenerbangan': 1,
                     'namaPemesan': 'John Doe',
                     'totalPrice': 5000000},
                    {'harga': 2600000,
                     'id': '3',
                     'jumlahTiket': 2,
                     'kodePenerbangan': 2,
                     'namaPemesan': 'Xyla',
                     'totalPrice': 5200000},
                    {'harga': 2800000,
                     'id': '5',
                     'jumlahTiket': 3,
                     'kodePenerbangan': 4,
                     'namaPemesan': 'Maharani',
                     'totalPrice': 8400000},
                    {'harga': 2700000,
                     'id': '6',
                     'jumlahTiket': 2,
                     'kodePenerbangan': 3,
                     'namaPemesan': 'Regina',
                     'totalPrice': 5400000}]}


# UPDATE KERANJANG

In [49]:
mutation_update_keranjang = """
mutation {
  updateKeranjang(input: {
    id: "1",
    namaPemesan: "RDW",
    kodePenerbangan: 1,
    jumlahTiket: 4
  }) {
    keranjang {
      id
      kodePenerbangan
      namaPemesan
      jumlahTiket
      harga
      totalPrice
    }
  }
}
"""


update_result = schema.execute(mutation_update_keranjang)
if update_result.errors:
    print("Errors occurred:")
    for error in update_result.errors:
        print(error)
else:
    print("Mutation result data:")
    pprint(update_result.data)

Mutation result data:
{'updateKeranjang': {'keranjang': {'harga': None,
                                   'id': '1',
                                   'jumlahTiket': 4,
                                   'kodePenerbangan': 1,
                                   'namaPemesan': 'RDW',
                                   'totalPrice': None}}}


In [50]:
query_keranjang_items = """
{
    keranjangItems {
        id
        kodePenerbangan
        namaPemesan
        jumlahTiket
        harga
        totalPrice
    }
}
"""

# Execute the query to fetch updated keranjang items
result = schema.execute(query_keranjang_items)

# Check for errors in the query execution
if result.errors:
    print("Errors occurred:")
    for error in result.errors:
        print(error)
else:
    print("Query result data:")
    pprint(result.data)


Query result data:
{'keranjangItems': [{'harga': 2500000,
                     'id': '1',
                     'jumlahTiket': 4,
                     'kodePenerbangan': 1,
                     'namaPemesan': 'RDW',
                     'totalPrice': 10000000},
                    {'harga': 2600000,
                     'id': '3',
                     'jumlahTiket': 2,
                     'kodePenerbangan': 2,
                     'namaPemesan': 'Xyla',
                     'totalPrice': 5200000},
                    {'harga': 2800000,
                     'id': '5',
                     'jumlahTiket': 3,
                     'kodePenerbangan': 4,
                     'namaPemesan': 'Maharani',
                     'totalPrice': 8400000},
                    {'harga': 2700000,
                     'id': '6',
                     'jumlahTiket': 2,
                     'kodePenerbangan': 3,
                     'namaPemesan': 'Regina',
                     'totalPrice': 5400000}]}
